### **마운트**


*   Google Colab에 마운트합니다.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


### **라이브러리 및 모듈 로드**

*   필요한 라이브러리들을 전부 설치하고 모듈을 로드합니다.

In [ ]:
!apt-get update &> /dev/null
!apt install chromium-chromedriver &> /dev/null
!cp /usr/lib/chromium-browser/chromedriver /usr/bin &> /dev/null
!pip install selenium &> /dev/null

from bs4 import BeautifulSoup
import requests
import urllib.request
from selenium import webdriver
import pandas as pd
import re
import time
import pickle

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

### **전역변수와 리스트 설정**

*   마지막 DataFrame에 들어갈 list들을 전역변수로 설정합니다.
*   키워드를 '저소득층'으로 설정하였고, 그 이유는 저소득층에 관련된 정책 혹은 지원, 시대별 저소득층 발생 원인 등에 대해서 알 수 있을 것 같아서 선정하였습니다.
*   추출 기간은 2011.01.01부터 2020.12.31입니다.

In [ ]:
url_list = []
title_list = []
year_list = []
month_list = []
day_list = []
content_list = []
news_list = []

keyword = '저소득층'

### **검색페이지 URL완성과 기사링크 접근하는 함수 만들기 + a) 제목, b) 날짜 추가**
* 한겨레 특성상 1000개의 기사를 넘게 가져오려 하면 같은 기사가 계속 출력되는 오류가 발생합니다.
* 이 오류를 없애기 위해 1년 단위로 쪼개서 기사를 가져오게 됩니다.

In [ ]:
def get_infos(news_num, year, keyword):
    iter_num = int(news_num / 10) + 1
    
    for i in range(iter_num):
        url = 'http://search.hani.co.kr/Search?command=query&keyword={}&media=news&submedia=&sort=d&period=all&datefrom={}.01.01&dateto={}.12.31&pageseq={}'.format(keyword, year, year, i)

        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')

        tmp = soup.find('ul', class_ = 'search-result-list')
        tmp2 = tmp.find_all('li')

        for j in tmp2:
            tmp3 = j.find('dt')
            infos = tmp3.find('a')
            url_list.append('http:' + infos.attrs['href'])
            title_list.append(infos.text.strip())

            tmp4 = j.find('dd', class_ = 'date')
            year_list.append(int(tmp4.text.strip()[:4]))
            month_list.append(int(tmp4.text.strip()[5:7]))
            day_list.append(int(tmp4.text.strip()[8:10]))

### **년도별 기사 수 가져오는 함수**
* 각 년도마다 기사가 몇개 있는지 가져오는 함수입니다.
* 상단의 get_infos를 같이 실행시켜 정보를 가져옵니다.

In [ ]:
def get_num_by_year(year, keyword):
    url = 'http://search.hani.co.kr/Search?command=query&keyword={}&media=news&submedia=&sort=d&period=all&datefrom={}.01.01&dateto={}.12.31&pageseq=0'.format(keyword, year, year)

    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    tmp = soup.find('span', class_ = 'total')
    news_num = int(re.findall('\d+', tmp.text)[0])

    get_infos(news_num, year, keyword)

### **기사링크 크롤링하여 c) 본문을 리스트에 넣는 함수 만들기**
* 기사를 가져오고, 한번에 가져오지 않고 pickle을 통해 데이터를 나눠서 가져오게 됩니다.

In [ ]:
def get_content(url_list, pickle_content):
    crawling_counter = 1

    for url in url_list:
        try:
            source_code_from_url = urllib.request.urlopen(url)
            soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding = 'utf-8')
            content_of_article = soup.select('div.text')

            content = ''

            for item in content_of_article:
                string_item = str(item.find_all(text = True))
                content += string_item

            pickle_content.append(content)
        
        except:
            pickle_content.append(' ')

        if crawling_counter % 100 == 0:
            print('{} out of {} crawling done...'.format(crawling_counter, len(url_list)))
        crawling_counter += 1


### **메인함수 실행**

*   전체 기사의 개수를 구하고, 링크 / 제목 / 날짜를 가져오고, 마지막으로 본문을 가져오는 함수를 실행합니다.
*   각 항목에 대해서 진행 상황을 간단하게 print문으로 출력하였습니다.
* 함수 실행에 시간이 너무 많이 걸려서, pickle을 사용해 데이터를 쪼개고 다시 가져오는 작업을 거치게 됩니다.

In [ ]:
start = time.time()

year_iters = range(2020, 2010, -1)

for year in year_iters:
    print('Year {} crawling...'.format(year))
    get_num_by_year(year, keyword)

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/hani_data.pickle', 'wb') as f:
    pickle.dump(url_list, f)
    pickle.dump(title_list, f)
    pickle.dump(year_list, f)
    pickle.dump(month_list, f)
    pickle.dump(day_list, f)

print('Total spend time : {} seconds'.format(time.time() - start))

Year 2020 crawling...
Year 2019 crawling...
Year 2018 crawling...
Year 2017 crawling...
Year 2016 crawling...
Year 2015 crawling...
Year 2014 crawling...
Year 2013 crawling...
Year 2012 crawling...
Year 2011 crawling...
Total spend time : 226.40969729423523 seconds


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/hani_data.pickle', 'rb') as f:
    url_list = pickle.load(f)
    title_list = pickle.load(f)
    year_list = pickle.load(f)
    month_list = pickle.load(f)
    day_list = pickle.load(f)

In [ ]:
start = time.time()

hani_content_1_1500 = []

print('Content number 1 to 1500 crawling start...')

get_content(url_list[:1500], hani_content_1_1500)

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/hani_content_1_1500.pickle', 'wb') as f:
    pickle.dump(hani_content_1_1500, f)

print('Content number 1 to 1500 spend time : {} seconds'.format(time.time() - start))

Content number 1 to 1500 crawling start...
100 out of 1500 crawling done...
200 out of 1500 crawling done...
300 out of 1500 crawling done...
400 out of 1500 crawling done...
500 out of 1500 crawling done...
600 out of 1500 crawling done...
700 out of 1500 crawling done...
800 out of 1500 crawling done...
900 out of 1500 crawling done...
1000 out of 1500 crawling done...
1100 out of 1500 crawling done...
1200 out of 1500 crawling done...
1300 out of 1500 crawling done...
1400 out of 1500 crawling done...
1500 out of 1500 crawling done...
Content number 1 to 1500 spend time : 1027.5361351966858 seconds


In [ ]:
start = time.time()

hani_content_1501_3000 = []

print('Content number 1501 to 3000 crawling start...')

get_content(url_list[1500:3000], hani_content_1501_3000)

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/hani_content_1501_3000.pickle', 'wb') as f:
    pickle.dump(hani_content_1501_3000, f)

print('Content number 1501 to 3000 spend time : {} seconds'.format(time.time() - start))

Content number 1501 to 3000 crawling start...
100 out of 1500 crawling done...
200 out of 1500 crawling done...
300 out of 1500 crawling done...
400 out of 1500 crawling done...
500 out of 1500 crawling done...
600 out of 1500 crawling done...
700 out of 1500 crawling done...
800 out of 1500 crawling done...
900 out of 1500 crawling done...
1000 out of 1500 crawling done...
1100 out of 1500 crawling done...
1200 out of 1500 crawling done...
1300 out of 1500 crawling done...
1400 out of 1500 crawling done...
1500 out of 1500 crawling done...
Content number 1501 to 3000 spend time : 1228.0559470653534 seconds


In [ ]:
start = time.time()

hani_content_3001_end = []

print('Content number 3001 to end crawling start...')

get_content(url_list[3000:], hani_content_3001_end)

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/hani_content_3001_end.pickle', 'wb') as f:
    pickle.dump(hani_content_3001_end, f)

print('Content number 3001 to end spend time : {} seconds'.format(time.time() - start))

Content number 3001 to end crawling start...
100 out of 1250 crawling done...
200 out of 1250 crawling done...
300 out of 1250 crawling done...
400 out of 1250 crawling done...
500 out of 1250 crawling done...
600 out of 1250 crawling done...
700 out of 1250 crawling done...
800 out of 1250 crawling done...
900 out of 1250 crawling done...
1000 out of 1250 crawling done...
1100 out of 1250 crawling done...
1200 out of 1250 crawling done...
Content number 3001 to end spend time : 964.4197661876678 seconds


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/hani_content_1_1500.pickle', 'rb') as f:
    tmp = pickle.load(f)
content_list += tmp

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/hani_content_1501_3000.pickle', 'rb') as f:
    tmp = pickle.load(f)
content_list += tmp

with open('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/hani_content_3001_end.pickle', 'rb') as f:
    tmp = pickle.load(f)
content_list += tmp

### **뉴스 리스트 작성**
* 어느 신문사에서 가져왔는지 알려주는 리스트입니다.

In [ ]:
news_list = ['한겨레'] * len(title_list)

### **쓸데없는 광고 제거**
* 광고에 들어가는 문구의 시작 지점을 잡고, 그 이후에 나오는 광고 문구들을 전부 제거합니다.

In [ ]:
for i in range(len(content_list)):
    adv_loc = content_list[i].find('news TOPSTORIES BOX st')
    if adv_loc != -1:
        content_list[i] = content_list[i][:adv_loc]

for i in range(len(content_list)):
    adv_loc_2 = content_list[i].find('news BOX st')
    if adv_loc_2 != -1:
        content_list[i] = content_list[i][:adv_loc_2]

### **저장한 리스트 DataFrame 작성**


*   각 list들을 DataFrame으로 묶는 구간입니다.

In [ ]:
total = pd.DataFrame({'title' : title_list, 'news' : news_list, 'year' : year_list, 'month' : month_list, 'day' : day_list, 'url' : url_list, 'content' : content_list})

In [ ]:
total.head(5)

,title,news,year,month,day,url,content
0,LH 땅 장사로 짓는 ‘찔끔’ 공공임대…정부 재정투입 늘려야,한겨레,2020,12,31,http://www.hani.co.kr/arti/economy/property/97...,"['\n', '\n', '\n', '\n', '\n', '문재인 대통령이 지난 12..."
1,31일 동정,한겨레,2020,12,30,http://www.hani.co.kr/arti/society/ngo/976614....,"['\n', '\n', '\n', '\n', '\n', '배우 최수종·하히라 부부...."
2,코로나19 충격에 0%대 금리시대...주가·집값은 급등,한겨레,2020,12,30,http://www.hani.co.kr/arti/economy/economy_gen...,"['\n', '\n', '\n', '\n', '\n', '올해는 코로나19로 인한 ..."
3,"금소법 3월 시행, 공모주 일반청약 물량 확대",한겨레,2020,12,30,http://www.hani.co.kr/arti/economy/finance/976...,"['\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n..."
4,‘한국형 실업부조’ 시동…내년 40만명에 6개월간 최대 300만원씩,한겨레,2020,12,28,http://www.hani.co.kr/arti/society/labor/97624...,"['\n', '\n', '\n', '\n', '\n', '이재갑 고용노동부 장관이 ..."


### **불필요한 행 제거**
* 한겨레는 가끔 X월 X일 알림, 또는 X일 동정 과 같은 제목의 기사들이 있습니다.
* 이 기사들은 조사하고 싶은 내용과 관련이 없다 판단하여, 이러한 제목 형태를 가진 기사들을 제거하였습니다.

In [ ]:
unused = []

for i in range(len(total['content'])):
    if total['content'][i] == ' ':
        unused.append(i)

for i in range(len(total['title'])):
    if ('월' in total['title'][i]) and ('일' in total['title'][i]) and ('알림' in total['title'][i]):
        unused.append(i)

for i in range(len(total['title'])):
    if ('일' in total['title'][i]) and ('동정' in total['title'][i]):
        unused.append(i)

unused = list(set(unused))
print(unused)
total = total.drop(index = unused, axis = 0)
total = total.dropna(axis = 0)
total = total.reset_index(drop = True)
total

[1, 520, 2572, 525, 1041, 2065, 1556, 2071, 1561, 3098, 2073, 29, 1566, 3615, 541, 34, 548, 1576, 1577, 1069, 1583, 3119, 2102, 1592, 2107, 1084, 573, 1596, 61, 1097, 1611, 76, 1612, 78, 3659, 1105, 595, 2135, 1112, 1113, 3162, 2139, 1629, 608, 609, 1121, 99, 1634, 103, 2664, 105, 617, 1642, 2151, 3183, 1136, 2160, 3703, 1657, 122, 1147, 636, 124, 1662, 2692, 3205, 1671, 1165, 1167, 1168, 1171, 2707, 3222, 1687, 2199, 665, 1179, 1183, 162, 1189, 167, 679, 169, 2217, 172, 1198, 2225, 179, 2741, 3262, 1215, 2239, 704, 194, 3269, 1223, 3785, 1226, 205, 1741, 1232, 2771, 2261, 1239, 728, 1752, 2777, 3288, 732, 2781, 2783, 3295, 2786, 2276, 1256, 746, 237, 2285, 2286, 2291, 245, 758, 2807, 1784, 763, 254, 767, 256, 3328, 769, 1797, 1286, 777, 780, 272, 784, 1296, 2320, 3347, 789, 278, 2326, 792, 2329, 795, 285, 1309, 287, 1311, 289, 291, 2342, 3366, 811, 2348, 2859, 1326, 2861, 1328, 814, 306, 819, 2355, 1333, 822, 815, 2866, 2877, 2366, 830, 2369, 2882, 1860, 3399, 1352, 1355, 1874, 2898, 

,title,news,year,month,day,url,content
0,LH 땅 장사로 짓는 ‘찔끔’ 공공임대…정부 재정투입 늘려야,한겨레,2020,12,31,http://www.hani.co.kr/arti/economy/property/97...,"['\n', '\n', '\n', '\n', '\n', '문재인 대통령이 지난 12..."
1,코로나19 충격에 0%대 금리시대...주가·집값은 급등,한겨레,2020,12,30,http://www.hani.co.kr/arti/economy/economy_gen...,"['\n', '\n', '\n', '\n', '\n', '올해는 코로나19로 인한 ..."
2,"금소법 3월 시행, 공모주 일반청약 물량 확대",한겨레,2020,12,30,http://www.hani.co.kr/arti/economy/finance/976...,"['\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n..."
3,‘한국형 실업부조’ 시동…내년 40만명에 6개월간 최대 300만원씩,한겨레,2020,12,28,http://www.hani.co.kr/arti/society/labor/97624...,"['\n', '\n', '\n', '\n', '\n', '이재갑 고용노동부 장관이 ..."
4,내년부터 증권거래세 0.23%로 낮아진다…신문구독료도 소득공제,한겨레,2020,12,28,http://www.hani.co.kr/arti/economy/economy_gen...,"['\n', '\n', '\n', '\n', '\n', '24일 오후 서울 중구 을..."
...,...,...,...,...,...,...,...
3985,[울림마당] 가르침이 행복한 선생님 많았으면…,한겨레,2011,1,5,http://www.hani.co.kr/arti/society/area/457296...,['\n 저는 지난해 7월부터 대전 용두동에 있는 비타민공부방에서 저소득층 초등학생...
3986,서울교육청 ‘교육소외학생 복지’ 초·중 전학교 실시,한겨레,2011,1,5,http://www.hani.co.kr/arti/society/schooling/4...,['\n 서울시교육청은 4일 저소득층을 위한 ‘교육복지투자우선지역 지원사업’과 ‘좋...
3987,서울시 “시의회 예산안 재의 요구”…시의회 “적법한 절차에 흠집내기”,한겨레,2011,1,5,http://www.hani.co.kr/arti/society/area/457264...,['\n 서울시는 지난달 30일 서울시의회가 수정·의결한 ‘2011년 서울시 예산안...
3988,"썬밸리호텔, 사랑의 실천",한겨레,2011,1,3,http://www.hani.co.kr/arti/economy/biznews/456...,['\n썬밸리골프그룹 이신근 회장을 대신해 여주썬밸리CC. 썬밸리호텔 이창기 사장이...


### **본문 및 제목 불필요한 내용 전처리**

*   먼저 영어, 숫자, 한글, 공백을 제외한 모든 기호를 제외합니다.
*   다음으로는 공백이 2번 이상인 경우가 있을 수 있다고 생각하여, 그것들을 모두 공백 한칸으로 처리합니다.
*   strip 함수를 통해 전체 content의 앞, 뒤로 있는 공백을 없앱니다.
*   처리가 잘 되었는지 확인을 위해 상위 5개의 결과값을 출력합니다.

In [ ]:
total['content'] = total['content'].str.replace(r'\\n', '')
total['content'] = total['content'].str.replace(r'\\r', '')
total['content'] = total['content'].str.replace(r'xa0', '')
total['content'] = total['content'].str.replace(r'[^a-zA-Z0-9\s가-힣]', ' ')
total['content'] = total['content'].str.replace(r'[ ]{2,}', ' ')
total['content'] = total['content'].str.strip()

total['content'][:5]

0    문재인 대통령이 지난 12월11일 화성 동탄의 공공임대주택을 둘러보고 있다 청와대사...
1    올해는 코로나19로 인한 실물경제 침체 속에서도 집값 고공행진이 이어졌다 서울 송파...
2    금융소비자의 권익을 제고할 금융소비자보호법 금소법 이 내년 3월25일에 시행되는 등...
3    이재갑 고용노동부 장관이 28일 오전 세종시 정부세종청사에서 국민취업지원제도 시행과...
4    24일 오후 서울 중구 을지로 하나은행 본점 딜링룸 현황판에 코스피 원 달러 환율 ...
Name: content, dtype: object

In [ ]:
total['title'] = total['title'].str.replace(r'\\n', '')
total['title'] = total['title'].str.replace(r'\\r', '')
total['title'] = total['title'].str.replace(r'xa0', '')
total['title'] = total['title'].str.replace(r'[^a-zA-Z0-9\s가-힣]', ' ')
total['title'] = total['title'].str.replace(r'[ ]{2,}', ' ')
total['title'] = total['title'].str.strip()

total['title'][:5]

0        LH 땅 장사로 짓는 찔끔 공공임대 정부 재정투입 늘려야
1           코로나19 충격에 0 대 금리시대 주가 집값은 급등
2               금소법 3월 시행 공모주 일반청약 물량 확대
3    한국형 실업부조 시동 내년 40만명에 6개월간 최대 300만원씩
4     내년부터 증권거래세 0 23 로 낮아진다 신문구독료도 소득공제
Name: title, dtype: object

### **XXX 기자 제거(완벽하지 않음)**

* content의 마지막 30개 character를 봐서, 그 안에 기자가 들어가 있으면 기사 마무리에 XXX 기자가 들어가 있다고 판단할 수 있습니다.
* 그럴 경우에는 '기자' 단어 앞 4글자(사람 이름)부터 끝까지 지우게 되면 쓸데없는 기자 이름 및 이메일을 삭제할 수 있을 것이라 생각합니다.

In [ ]:
for i in range(len(total['content'])):
    gija_loc = total['content'][i].rfind('기자')
    if len(total['content'][i]) - gija_loc < 30:
        total['content'][i] = total['content'][i][:gija_loc - 4]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


### **CSV 파일로 저장**

*   google drive를 사용하여 제 드라이브의 폴더에 저소득층_한겨레라는 파일 이름으로 csv 파일을 저장하였습니다.


In [ ]:
total.to_csv('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/저소득층_한겨레.csv', encoding="utf-8-sig")

### **한겨레, 경향 신문 합치기**

* 경향2 코드를 통해 나온 CSV와 본 파일을 통해 나온 CSV를 통합합니다. 쓸데없는 열을 지우고 index를 초기화해서 합치게 됩니다.

In [ ]:
hani_total = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/저소득층_한겨레.csv')
khan_total = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/저소득층_경향.csv')

hani_khan_total = pd.concat([hani_total, khan_total])

hani_khan_total = hani_khan_total.drop([hani_khan_total.columns[0]], axis = 1)
hani_khan_total = hani_khan_total.reset_index()
hani_khan_total = hani_khan_total.drop([hani_khan_total.columns[0]], axis = 1)

hani_khan_total

,title,news,year,month,day,url,content
0,LH 땅 장사로 짓는 찔끔 공공임대 정부 재정투입 늘려야,한겨레,2020,12,31,http://www.hani.co.kr/arti/economy/property/97...,문재인 대통령이 지난 12월11일 화성 동탄의 공공임대주택을 둘러보고 있다 청와대사...
1,코로나19 충격에 0 대 금리시대 주가 집값은 급등,한겨레,2020,12,30,http://www.hani.co.kr/arti/economy/economy_gen...,올해는 코로나19로 인한 실물경제 침체 속에서도 집값 고공행진이 이어졌다 서울 송파...
2,금소법 3월 시행 공모주 일반청약 물량 확대,한겨레,2020,12,30,http://www.hani.co.kr/arti/economy/finance/976...,금융소비자의 권익을 제고할 금융소비자보호법 금소법 이 내년 3월25일에 시행되는 등...
3,한국형 실업부조 시동 내년 40만명에 6개월간 최대 300만원씩,한겨레,2020,12,28,http://www.hani.co.kr/arti/society/labor/97624...,이재갑 고용노동부 장관이 28일 오전 세종시 정부세종청사에서 국민취업지원제도 시행과...
4,내년부터 증권거래세 0 23 로 낮아진다 신문구독료도 소득공제,한겨레,2020,12,28,http://www.hani.co.kr/arti/economy/economy_gen...,24일 오후 서울 중구 을지로 하나은행 본점 딜링룸 현황판에 코스피 원 달러 환율 ...
...,...,...,...,...,...,...,...
9334,배고플 때 받은 자활의 떡 이젠 이웃들과 나눠야죠,경향,2011,1,4,http://news.khan.co.kr/kh_news/khan_art_view.h...,사회적기업 꿈꾸는 떡집 떡찌니 석창근씨 가족서울 도곡동의 떡집 떡찌니 는 요즘 한창...
9335,사설 보편적 무상급식의 원년 으로 기록될 올해,경향,2011,1,3,http://news.khan.co.kr/kh_news/khan_art_view.h...,전국 16개 시 도 가운데 올해 보편적 무상급식 무상급식 을 실시하는 곳이 많다고 ...
9336,썬밸리골프그룹 사랑을 나누다,경향,2011,1,3,http://news.khan.co.kr/kh_news/khan_art_view.h...,세밑 한파가 기승을 부린 지난 주 썬밸리골프그룹과 동광종합토건 주 회장 이신근 이 ...
9337,눈칫밥 없는 무상급식 원년 새 이정표,경향,2011,1,2,http://news.khan.co.kr/kh_news/khan_art_view.h...,시 도별 어떻게 시작하나 무상급식 원년 올해는 모든 학생이 차별 없이 무상으로 학교...


### **최종 파일 csv 파일로 저장**

In [ ]:
hani_khan_total.to_csv('/content/drive/MyDrive/Colab Notebooks/소셜빅데이터분석/저소득층_한겨레_경향_통합.csv', encoding="utf-8-sig")